In [6]:
#Day 10 AOC 23
with open("input.txt", "r") as file:
    puzzle_in = file.read().splitlines()
print(puzzle_in[:10])

with open("example.txt", "r") as file:
    example = file.read().splitlines()
print(example[:10])

with open("example2.txt", "r") as file:
    example2 = file.read().splitlines()
print(example2[:10])

['%dj -> fj, jn', '%xz -> cm', '%fn -> rj', '%fv -> nt, zp', '%ls -> ph, cf', '%rk -> zp, tp', '&jn -> km, cr, vz', '%nh -> ph, ls', '%tx -> gb', '%xg -> dv, zp']
['broadcaster -> a, b, c', '%a -> b', '%b -> c', '%c -> inv', '&inv -> a']
['broadcaster -> a', '%a -> inv, con', '&inv -> b', '%b -> con', '&con -> output']


In [27]:
class con_mod():
    def __init__(self, name, dests):
        self.targets = dests
        self.name = name
        self.inputs = {}
        

    def set_feedin(self, inputs):
        self.inputs = {mod_name: False for mod_name in inputs}

    def receive_send(self, signal, sender):
        # update appropriate memory slot
        self.inputs[sender] = signal

        # send out pulse
        pulse = not(all(self.inputs.values()))
        
        return [(pulse, dest, self.name) for dest in self.targets]

class flip_mod():
    def __init__(self, name, dests):
        self.on = False
        self.name = name
        self.targets = dests

    def is_on(self):
        return self.on

    def receive_send(self, pulse, sender):
        if pulse: return [] #pulse == True, the pulse is high
        self.on = not self.on
        # if it becomes on, it send a high pulse. If it becomes off, send low pulse
        return [(self.is_on(), dest, self.name) for dest in self.targets] 


# parse input
def parse_p1(data):
    modules = {}
    broadcast = ""
    feedins = {}
    conjuncts = []
    for line in data:
        name, targets = line.split(' -> ')
        targets = targets.split(', ')
        if name == 'broadcaster':
            broadcast = targets
            continue
        module_type, name = name[0], name[1:]
        if module_type == '%':
            modules[name] = flip_mod(name, targets)
        elif module_type == '&':
            modules[name] = con_mod(name, targets)
            conjuncts.append(name)
        else:
            KeyError("unknown module type")
        for target in targets:
            feedins[target] = feedins.get(target, []) + [name]

    for conjunction_mod in conjuncts:
        modules[conjunction_mod].set_feedin(feedins[conjunction_mod])
        
    return modules, broadcast


In [43]:
#part 1
modules, broadcast = parse_p1(puzzle_in)

def press(modules, broadcast):
    high_pulses = 0
    low_pulses = 1 + len(broadcast)
    queue = [(False, d, 'broadcast') for d in broadcast]
    
    while queue:
        sig, destination, sender = queue.pop(0)

        # print(f"{sender} -- {'high' if sig else ' low '} --> {destination}")
        
        module = modules.get(destination, None)
        if not module: continue

        new_pulses = module.receive_send(sig, sender)
        queue.extend(new_pulses)

        if new_pulses and new_pulses[0][0]:
            high_pulses += len(new_pulses)
        elif new_pulses:
            low_pulses += len(new_pulses)

    return modules, high_pulses, low_pulses
    
ht = lt = 0
hl, ll = [], []
for i in range(1000):   
    # print(f"\nBROADCAST {i}\n")
    modules, h, l = press(modules, broadcast)
    hl.append(h)
    ll.append(h)
    ht += h
    lt += l

print(ht, lt, ht*lt)
print(ll)
print(hl)

#example1:
#given - 32000000
#found - 32000000
#example2:
#given - 11687500
#found - 11687500
#puzzle_in:
#a1 - 832957356 correct :D


45644 18249 832957356
[33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 95, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 106, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 95, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 125, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 95, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 106, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 95, 33, 44, 33, 64, 33, 44, 33, 66, 33, 44, 33, 64, 33, 44, 33, 86, 33, 44, 33, 64